# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [5]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sponge_bob_magic.datasets.movielens import MovieLens
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic import metrics

from sponge_bob_magic.models.pop_rec import PopRec
from sponge_bob_magic.models.als_rec import ALSRec
from sponge_bob_magic.models.knn_rec import KNNRec
from sponge_bob_magic.models.lightfm_rec import LightFMRec

from sponge_bob_magic.scenarios.main_scenario.main_factory import MainScenarioFactory
from sponge_bob_magic.session_handler import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [6]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [7]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [8]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [9]:
pop_rec = PopRec()
als_rec = ALSRec()
knn_rec = KNNRec()
lightfm_rec = LightFMRec()

In [17]:
log_bydate_splitter = log_splitter.DateSplitter(
    test_start=datetime(2007, 1, 1),
    drop_cold_items=True,
    drop_cold_users=True
)
log_random_splitter = log_splitter.RandomSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)
log_cold_splitter = log_splitter.ColdUsersSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True
)
user_random_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
    
)
user_bydate_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)

### Обучение сценария <a name="fit-scenario"></a>

In [18]:
results = None

In [26]:
scenario = MainScenarioFactory().get(
    splitter=user_random_splitter,
    recommender=lightfm_rec,
    criterion=metrics.HitRate(),
    metrics={
        metrics.NDCG(): [10,5,3],
        metrics.Precision(): [10,5,3],
        metrics.MAP(): [10,5,3],
        metrics.Recall(): [10,5,3],
        metrics.Surprisal(): [10,5,3],
    },
    fallback_rec=pop_rec,
)

In [27]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [44]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

20-Feb-20 17:10:23, root, DEBUG: Деление лога на обучающую и тестовую выборку
20-Feb-20 17:10:39, root, DEBUG: Длина трейна и теста: (84013, 15944)
20-Feb-20 17:10:39, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490
20-Feb-20 17:10:40, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355
20-Feb-20 17:10:40, root, DEBUG: Обучение и предсказание дополнительной модели
20-Feb-20 17:10:40, root, DEBUG: Проверка датафреймов
20-Feb-20 17:10:40, root, DEBUG: Предварительная стадия обучения (pre-fit)
20-Feb-20 17:10:41, root, DEBUG: Среднее количество items у каждого user: 90
20-Feb-20 17:10:44, root, DEBUG: Основная стадия обучения (fit)
20-Feb-20 17:10:44, root, DEBUG: Проверка датафреймов
20-Feb-20 17:10:45, root, DEBUG: Количество items после фильтрации: 100
20-Feb-20 17:10:48, root, DEBUG: Пре-фит модели
20-Feb-20 17:10:48, root, DEBUG: -------------
20-Feb-20 17:10:48, root, DEBUG: Оптимизация параметров
20-Feb-20 17:10:48, root, DEBUG: Максимальное количество по

In [45]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

number                state     value             datetime_start  \
                                                                     
0  0      TrialState.COMPLETE  0.836735 2020-02-20 17:10:48.859180   
1  1      TrialState.COMPLETE  0.867347 2020-02-20 17:11:40.762658   

           datetime_complete params  \
                               rank   
0 2020-02-20 17:11:40.683705  40      
1 2020-02-20 17:12:32.160223  17      

                                                                 user_attrs  \
                                                                        MAP   
0  {5: 0.4297619047619051, 10: 0.49487631921627334, 3: 0.38180272108843566}   
1  {5: 0.43971258503401384, 10: 0.504489696936976, 3: 0.38962585034013664}    

                                                                             \
                                                                  Precision   
0  {5: 0.2285714285714286, 10: 0.23326530612244895, 3: 0.22721088435374137}   
1  {5: 0.23795918367346927, 10: 0.23551020408163276, 3: 0.2374149659863945}   

                                                                               \
                                                                       Recall   
0  {5: 0.04987595025655177, 10: 0.10147207503889259, 3: 0.030540965436675723}   
1  {5: 0.05385651676115328, 10: 0.10134954487146615, 3: 0.0329999972079278}     

                                                                            \
                                                                 Surprisal   
0  {5: 0.371652519714292, 10: 0.3837529377110808, 3: 0.36499400049430475}    
1  {5: 0.3646961310213943, 10: 0.37558379967093186, 3: 0.3558428405757786}   

                                                                              \
                                                                        nDCG   
0  {5: 0.23541426612793626, 10: 0.24372047036203845, 3: 0.236962763810413}     
1  {5: 0.24345332212101245, 10: 0.24760044534324474, 3: 0.24520599011723207}   

  system_attrs  
       _number  
0  0            
1  1

### Получение рекомендаций <a name="predict-scenario"></a>

In [46]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

20-Feb-20 17:13:40, root, DEBUG: Проверка датафреймов
20-Feb-20 17:13:40, root, DEBUG: Предварительная стадия обучения (pre-fit)
20-Feb-20 17:13:41, root, DEBUG: Основная стадия обучения (fit)
20-Feb-20 17:13:41, root, DEBUG: Построение модели LightFM
20-Feb-20 17:13:42, root, DEBUG: Проверка датафреймов
20-Feb-20 17:13:42, root, DEBUG: Выделение дефолтных юзеров
20-Feb-20 17:13:42, root, DEBUG: Выделение дефолтных айтемов


In [47]:
recs.show()

+-------+-------+--------------------+----------+
|user_id|item_id|           relevance|   context|
+-------+-------+--------------------+----------+
|     91|    199|  0.5322619080543518|no_context|
|     91|    194| 0.20798559486865997|no_context|
|     91|    661|  0.1420615166425705|no_context|
|     91|    511| 0.09485377371311188|no_context|
|     91|    205|-0.00296466005966...|no_context|
|     91|    435|-0.08984896540641785|no_context|
|     91|    510|-0.15943878889083862|no_context|
|     91|    480|-0.17838163673877716|no_context|
|     91|    135|-0.23543228209018707|no_context|
|     91|    520| -0.2369796484708786|no_context|
|    152|    237|-0.00513449730351...|no_context|
|    152|     15|-0.14510269463062286|no_context|
|    152|    125|-0.49600809812545776|no_context|
|    152|     88| -0.5041524767875671|no_context|
|    152|      1| -0.5674199461936951|no_context|
|    152|    111| -0.5692919492721558|no_context|
|    152|     69| -0.6893871426582336|no_context|
